## UC1 - Building with AzureOpenAI realtime API 


The GPT-4o audio realtime API is designed to handle real-time, low-latency conversational interactions, making it a great fit for use cases involving live interactions between a user and a model, such as customer support agents, voice assistants, and real-time translators.

Azure OpenAI GPT-4o real-time API supports low-latency, "speech in, speech out" conversational interactions.
It works with text messages, function tool calling, and many other existing capabilities from other endpoints like /chat/completions.
Is a great fit for support agents, assistants, translators, and other use cases that need highly responsive back-and-forth with a user.

Please review the [GPT-4o Realtime API documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/realtime-audio-quickstart?pivots=ai-foundry-portal)



## Deploying the model
On your Azure AI Foundry Portal choose Models + Endpoints, Deploy Mode / Deploy a base Model and finally choose gpt-4o-realtime as the model to be deployed.

![Deploying GPT-4o Realtime API](./images/p1.png)

Next, edit the deployment to increase the number of calls per minute to the max value.
![Increase calls per minute to the model](./images/p2.png)



## Test GPT-4o Azure OpenAI API Endpoint with Completions API 

Once a text-based model such as gpt-4o is deployed, let's use it to generate completions for a given a simple prompt.
This way we will confirm the API key and endpoint URL are configured correctly.
Move on to the next section when a successful generation is completed.

In [ ]:
!pip install os
!pip install python-dotenv
!pip install openai

In [ ]:
import dotenv
import os 
from openai import AzureOpenAI

dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
        
# Send a completion call to generate an answer
completion = client.chat.completions.create(
    model="gpt-4o",
    messages = [
        {
        "role": "system",
        "content": "You are an MIT PhD in Physics, specializing in quantum physics."
        },
        {
        "role": "user",
        "content": "What is a black hole?"
        }
    ]
    # max_tokens=4096
)

#print(completion.model_dump_json(indent=2))
content = completion.choices[0].message.content
print(content)
print(len(content))

## Connecting to and authenticating with /realtime

/realtime is built on the [WebSockets API](https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API) to facilitate fully asynchronous streaming communication between the end user and model. \

The /realtime API requires an existing Azure OpenAI resource endpoint in a supported region. 
A full request URI can be constructed by concatenating:

1. The secure WebSocket (wss://) protocol \
2. Your Azure OpenAI resource endpoint hostname, e.g. my-aoai-resource.openai.azure.com \
3. The openai/realtime API path \
4. An api-version query string parameter for a supported API version -- initially, 2024-10-01-preview \
5. A deployment query string parameter with the name of your gpt-4o-realtime-preview model deployment \

Combining into a full example, the following could be a well-constructed /realtime request URI:
For example in my case the the full request URI is:

```plaintext
wss://aoai-ep-swedencentral02.openai.azure.com/openai/realtime?api-version=2024-10-01-preview&deployment=gpt-4o-realtime-preview

```


In [31]:
!pip install websockets
!pip install asyncio
!pip install nest_asyncio


In [ ]:
import websockets
import asyncio
import os
from dotenv import load_dotenv
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Load environment variables from .env
load_dotenv()

# Retrieve API key and WSS URL
api_key = os.getenv("AZURE_OPENAI_API_KEY")
wss_url = "wss://aoai-ep-swedencentral02.openai.azure.com/openai/realtime?api-version=2024-10-01-preview&deployment=gpt-4o-realtime-preview"

if not api_key:
    raise ValueError("API_KEY not found. Ensure it is set in the .env file.")

async def confirm_connection():
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    try:
        async with websockets.connect(wss_url, extra_headers=headers) as websocket:
            print("Successfully connected to the /realtime endpoint!")
    except Exception as e:
        print(f"Connection failed: {e}")

# Run the WebSocket client in Jupyter Notebook
await confirm_connection()


ValueError: API_KEY not found. Ensure it is set in the .env file.